# 1. Blocker Fraud Company
 <font size="2"> Dados e problema negócio hipotéticos</font>
 
A Blocker Fraud Company é uma empresa especializada na detecção de fraudes em transações financeiras feitas atravś de dispositivos móveis.

O modelo de negócio da empresa é do tipo Serviço com a monetização feita por performance do serviço prestado, ou seja, o usuário paga uma taxa fixa sobre o sucesso da detecção de fraude das transações.

## 1.1. Estratégia de expansão

A Blocker Fraud Company está em fase de expansão no Brasil e para adquirir clientes mais rapidamente, ela adotou uma estratégia muito agressiva. A estratégia funciona da seguinte forma:

1. A empresa vai receber 25% do valor de cada transação detectada verdadeiramente como fraude.
2. A empresa vai receber 5% do valor de cada transação detectada como fraude, porém a transação é verdadeiramente legítima.
3. A empresa vai devolver 100% do valor para o cliente, a cada transação detectada como legítima, porém a transação é verdadeiramente uma fraude.

Com essa estratégia agressiva a empresa assume os riscos em falhar na detecção de fraude e é remunerada na detecção assertiva das fraudes.

## 1.2. Objetivo

Criar um modelo de alta precisão e acurácia na detecção de fraudes de transações feitas através de dispositivos móveis.

### 1.2.1. Entregáveis

Modelo em produção no qual seu acesso será feito via API, ou seja, os clientes enviarão suas transações via API para que o seu modelo as classifique como fraudulentas ou legítimas.

Além disso, você precisará entregar um relatório reportando a performance e os resultados do seu modelo em relação ao lucro e prejuízo que a empresa terá ao usar o modelo que você produziu. No seu relatório deve conter as respostas para as seguintes perguntas:

1. Qual a Precisão e Acurácia do modelo?
2. Qual a Confiabilidade do modelo em classificar as transações como legítimas ou fraudulentas?
3. Qual o Faturamento Esperado pela Empresa se classificarmos 100% das transações com o modelo?
4. Qual o Prejuízo Esperado pela Empresa em caso de falha do modelo?
5. Qual o Lucro Esperado pela Blocker Fraud Company ao utilizar o modelo?


# 2. Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

#display
pd.set_option('display.float_format', lambda x: '%.5f' % x)

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

from sklearn import model_selection
from sklearn import ensemble
from sklearn import tree
from sklearn import linear_model
from sklearn import pipeline
from sklearn import preprocessing

from feature_engine import imputation
from feature_engine import encoding
from feature_engine import discretisation
from feature_engine import wrappers 

## 2.1. Utils functions

# 3. Data

## 3.1. Load dataset

In [2]:
df = pd.read_parquet('../data/raw/df_applied_features.parquet.gzip') 

In [3]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,is_high_amount,is_M_customer,is_new_balance_orig_zero,is_new_balance_dest_zero,diff_orig,diff_dest,is_orig_old_balance_greater_new,is_dest_old_balance_greater_new
0,1,PAYMENT,9839.64000,C1231006815,170136.00000,160296.36000,M1979787155,0.00000,0.00000,0,0,0,1,0,1,9839.64000,0.00000,1,0
1,1,PAYMENT,1864.28000,C1666544295,21249.00000,19384.72000,M2044282225,0.00000,0.00000,0,0,0,1,0,1,1864.28000,0.00000,1,0
2,1,TRANSFER,181.00000,C1305486145,181.00000,0.00000,C553264065,0.00000,0.00000,1,0,0,0,1,1,181.00000,0.00000,1,0
3,1,CASH_OUT,181.00000,C840083671,181.00000,0.00000,C38997010,21182.00000,0.00000,1,0,0,0,1,1,181.00000,21182.00000,1,1
4,1,PAYMENT,11668.14000,C2048537720,41554.00000,29885.86000,M1230701703,0.00000,0.00000,0,0,0,1,0,1,11668.14000,0.00000,1,0


# 4. Data preparation

Para os dados númericos iremos atuar como a discretisation, que é o processo de transformar variáveis contínuas em discretas criando intervalos contíguos que espalha os intervalos dos valores das variáveis, criando assim uma variável mais normal.

Além disso, também trataremos as variáveis categóricas, algumas serão transformadas em raras auxiliando o modelo.

## 4.1. Define feature and target

In [4]:
features = df.drop('isFraud', axis=1).columns.tolist()
target = 'isFraud'

## 4.2. Split data into train and valid

In [5]:
X = df[features]
y = df[target].values

X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X,
                                                                    y,
                                                                    random_state=42,
                                                                    test_size=0.3,
                                                                     stratify=y)

In [6]:
cols_to_include = ['isFlaggedFraud', 'is_high_amount', 'is_M_customer', 
                   'is_new_balance_orig_zero', 'is_orig_old_balance_greater_new',
                   'is_new_balance_dest_zero', 'is_dest_old_balance_greater_new']

columns_drop = ['nameOrig', 'nameDest']

for col in cols_to_include:
    X_train[col] = X_train[col].astype('category')
    X_valid[col] = X_valid[col].astype('category')
    
X_train.drop(columns_drop, axis=1, inplace=True)
X_valid.drop(columns_drop, axis=1, inplace=True)

## 4.3. Pipeline

In [7]:
cat_features = ['is_new_balance_dest_zero',
                'type',
                'is_orig_old_balance_greater_new',
                'is_M_customer',
                'is_high_amount',
                'is_new_balance_orig_zero',
                'is_dest_old_balance_greater_new',
                'isFlaggedFraud']
num_features = ['amount',
                'oldbalanceOrg',
                'newbalanceOrig',
                'oldbalanceDest',
                'newbalanceDest',
                'diff_orig',
                'diff_dest']

In [8]:
rf_clf = ensemble.RandomForestClassifier(n_jobs=-1,
                                         random_state=42)

In [9]:
cat_enc = encoding.CountFrequencyEncoder(variables=cat_features)
rare_enc = encoding.RareLabelEncoder(variables=['type', 'isFlaggedFraud'])
numerical_enc = wrappers.SklearnTransformerWrapper(transformer = preprocessing.StandardScaler(),
                                                   variables = num_features)

In [10]:
pipe = pipeline.Pipeline(steps = [("enconding_rare", rare_enc),
                                  ("enconding_cat", cat_enc),
                                  ("enconding_num", numerical_enc),
                                  ("model", rf_clf)])

In [11]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('enconding_rare',
                 RareLabelEncoder(variables=['type', 'isFlaggedFraud'])),
                ('enconding_cat',
                 CountFrequencyEncoder(variables=['is_new_balance_dest_zero',
                                                  'type',
                                                  'is_orig_old_balance_greater_new',
                                                  'is_M_customer',
                                                  'is_high_amount',
                                                  'is_new_balance_orig_zero',
                                                  'is_dest_old_balance_greater_new',
                                                  'isFlaggedFraud'])),
                ('enconding_num',
                 SklearnTransformerWrapper(transformer=StandardScaler(),
                                           variables=['amount', 'oldbalanceOrg',
                                                      'newbalanceOrig',
       

## 4.4. Persist pipeline

In [13]:
joblib.dump(pipe, '../models/model_cicle_1.pkl', compress = 1)

['../models/model_cicle_1.pkl']

# 5. Conclusion

Nessa etapa fizemos o preprocessamento e treinamento do modelo que será utilizado adiante, para tal definimos quais são as variável categóricas e as númericas e dividimos os dados em treino e validação. Vimos que a variável alvo é bem desbalanceada, porém por enquanto não iremos atuar nesse quesito, deixando para os próximos ciclos.

Na etapa anterior verificamos algumas variáveis que continha poucos valores distintos (type e isFlaggedFraud), portanto iremos transformar esses valores em Raro para auxiliar o modelo nas observações que possuem uma porcentagem baixo de categorias. Para as restantes das categóricas, usamos `CountFrequencyEncoder` onde substitui as categorias pela contagem ou pela porcentagem de observações por categoria da variável.

As númericas, que vimos serem contínuas, será transformada pelo `StandardScaler` onde é feito a transformação dessas variável para uma média igual a zero e escala a variância a uma unidade. Isso significa que para cada feature, a média seria 0, e o Desvio Padrão seria 1

O algoritmo escolhido para as predições é o RandomForest, é um bom algoritmo para esse primeiro ciclo para entendermos se é possível utilizar ele para as predições, e além disso podemos extrair informações valiosas para o negócio, esse modelo é possível saber quais as variáveis que levaram à fraude.